Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
############################################################
########## Neural Networks with L2 regularization ##########
############################################################
batch_size = 128
hidden_nodes = 1024
beta = 0.003
graph = tf.Graph()

def forward_prop(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    result = tf.matmul(relus, weights2) + biases2
    return result

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits = forward_prop(tf_train_dataset, weights1, biases1, weights2, biases2)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1332.392090
Minibatch accuracy: 7.0%
Validation accuracy: 40.7%
Minibatch loss at step 500: 208.911194
Minibatch accuracy: 80.5%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 46.341267
Minibatch accuracy: 81.2%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 10.768469
Minibatch accuracy: 85.9%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 2.969135
Minibatch accuracy: 82.8%
Validation accuracy: 86.0%
Minibatch loss at step 2500: 1.161974
Minibatch accuracy: 84.4%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 0.734154
Minibatch accuracy: 84.4%
Validation accuracy: 85.5%
Test accuracy: 91.8%


In [7]:
################################################################
########## Logistic Regression with L2 regularization ##########
################################################################
batch_size = 128
beta = 0.003
graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ \
         beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 25.731052
Minibatch accuracy: 14.1%
Validation accuracy: 13.2%
Minibatch loss at step 500: 2.426343
Minibatch accuracy: 79.7%
Validation accuracy: 76.4%
Minibatch loss at step 1000: 1.083296
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.704010
Minibatch accuracy: 84.4%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.881944
Minibatch accuracy: 75.0%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 0.805078
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.703773
Minibatch accuracy: 79.7%
Validation accuracy: 80.6%
Test accuracy: 87.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
############################################################
########## Neural Networks with L2 regularization ##########
############################################################
batch_size = 128
hidden_nodes = 1024
beta = 0.003
graph = tf.Graph()

train_dataset_size = batch_size * 8 # Reducing the training size
train_dataset_small = train_dataset[:train_dataset_size,:]
train_labels_small = train_labels[:train_dataset_size,:]

def forward_prop(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    result = tf.matmul(relus, weights2) + biases2
    return result

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits = forward_prop(tf_train_dataset, weights1, biases1, weights2, biases2)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))

num_steps = 3001

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset_small[offset:(offset + batch_size), :]
            batch_labels = train_labels_small[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1320.519043
Minibatch accuracy: 5.5%
Validation accuracy: 29.7%
Minibatch loss at step 500: 210.220352
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 1000: 46.857590
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 10.499228
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 2.461399
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 0.686993
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 0.279096
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Test accuracy: 86.9%


In [12]:
################################################################
########## Logistic Regression with L2 regularization ##########
################################################################
batch_size = 128
beta = 0.003
graph = tf.Graph()

train_dataset_size = batch_size * 10 # Reducing the training size
train_dataset_small = train_dataset[:train_dataset_size,:]
train_labels_small = train_labels[:train_dataset_size,:]

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+ \
         beta * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 25.169205
Minibatch accuracy: 10.2%
Validation accuracy: 10.8%
Minibatch loss at step 500: 2.135530
Minibatch accuracy: 95.3%
Validation accuracy: 71.3%
Minibatch loss at step 1000: 0.677691
Minibatch accuracy: 99.2%
Validation accuracy: 74.7%
Minibatch loss at step 1500: 0.403549
Minibatch accuracy: 97.7%
Validation accuracy: 75.9%
Minibatch loss at step 2000: 0.362383
Minibatch accuracy: 99.2%
Validation accuracy: 76.2%
Minibatch loss at step 2500: 0.323515
Minibatch accuracy: 99.2%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 0.287390
Minibatch accuracy: 97.7%
Validation accuracy: 76.4%
Test accuracy: 84.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [13]:
############################################################
########## Neural Networks with L2 regularization ##########
############################################################
batch_size = 128
hidden_nodes = 1024
beta = 0.003
graph = tf.Graph()
keep_prob = 0.5

def forward_prop(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    result = tf.matmul(relus, weights2) + biases2
    return result

def forward_prop_training(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    dropped = tf.nn.dropout(relus, keep_prob)
    result = tf.matmul(dropped, weights2) + biases2
    return result



with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits = forward_prop_training(tf_train_dataset, weights1, biases1, weights2, biases2)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1351.855957
Minibatch accuracy: 10.9%
Validation accuracy: 25.1%
Minibatch loss at step 500: 211.024231
Minibatch accuracy: 77.3%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 46.716511
Minibatch accuracy: 78.1%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 10.847451
Minibatch accuracy: 84.4%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 3.082342
Minibatch accuracy: 78.1%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 1.238251
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 0.818321
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Test accuracy: 91.4%


In [14]:
############################################################
########## Neural Networks with L2 regularization ##########
##########           Extreme Overfitting          ##########
############################################################
batch_size = 128
hidden_nodes = 1024
beta = 0.003
graph = tf.Graph()
keep_prob = 0.5

train_dataset_size = batch_size * 8 # Reducing the training size
train_dataset_small = train_dataset[:train_dataset_size,:]
train_labels_small = train_labels[:train_dataset_size,:]

def forward_prop(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    result = tf.matmul(relus, weights2) + biases2
    return result

def forward_prop_training(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    dropped = tf.nn.dropout(relus, keep_prob)
    result = tf.matmul(dropped, weights2) + biases2
    return result

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits = forward_prop_training(tf_train_dataset, weights1, biases1, weights2, biases2)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))

num_steps = 3001

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset_small[offset:(offset + batch_size), :]
            batch_labels = train_labels_small[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1512.900146
Minibatch accuracy: 7.0%
Validation accuracy: 35.9%
Minibatch loss at step 500: 211.517014
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 47.180443
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 10.568097
Minibatch accuracy: 100.0%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 2.484183
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 2500: 0.726649
Minibatch accuracy: 99.2%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 0.312441
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Test accuracy: 86.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [15]:
############################################################
########## Neural Networks with L2 regularization ##########
############################################################
##########        Decaying Learning Rate          ##########
batch_size = 128
hidden_nodes = 1024
beta = 0.003
graph = tf.Graph()
keep_prob = 0.5

def forward_prop(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    result = tf.matmul(relus, weights2) + biases2
    return result

def forward_prop_training(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    dropped = tf.nn.dropout(relus, keep_prob)
    result = tf.matmul(dropped, weights2) + biases2
    return result



with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes]))
  
  weights2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  logits = forward_prop_training(tf_train_dataset, weights1, biases1, weights2, biases2)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
    
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step,decay_steps = 500, decay_rate = 0.9)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1412.786133
Minibatch accuracy: 13.3%
Validation accuracy: 27.8%
Minibatch loss at step 500: 722.530273
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Minibatch loss at step 1000: 549.713745
Minibatch accuracy: 78.1%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 432.261169
Minibatch accuracy: 80.5%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 350.743591
Minibatch accuracy: 72.7%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 288.833862
Minibatch accuracy: 75.0%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 244.483322
Minibatch accuracy: 73.4%
Validation accuracy: 83.5%
Test accuracy: 90.1%


In [16]:
############################################################
########## Neural Networks with L2 regularization ##########
############################################################
##########           Go Wider                     ##########  

batch_size = 128
beta = 0.003
graph = tf.Graph()
keep_prob = 0.5
hidden_layer = 16384
def forward_prop(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    result = tf.matmul(relus, weights2) + biases2
    return result

def forward_prop_training(dataset, weights1, biases1, weights2, biases2):
    relus = tf.nn.relu(tf.matmul(dataset, weights1) + biases1)
    dropped = tf.nn.dropout(relus, keep_prob)
    result = tf.matmul(dropped, weights2) + biases2
    return result



with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer]))
  biases1 = tf.Variable(tf.zeros([hidden_layer]))
  
  weights2 = tf.Variable(tf.truncated_normal([hidden_layer, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = forward_prop_training(tf_train_dataset, weights1, biases1, weights2, biases2)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step,decay_steps = 500, decay_rate = 0.9)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2))
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16879.451172
Minibatch accuracy: 10.2%
Validation accuracy: 37.6%
Minibatch loss at step 500: 11464.883789
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 8895.655273
Minibatch accuracy: 83.6%
Validation accuracy: 85.3%
Minibatch loss at step 1500: 7063.211426
Minibatch accuracy: 81.2%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 5709.463867
Minibatch accuracy: 84.4%
Validation accuracy: 87.3%
Minibatch loss at step 2500: 4728.709961
Minibatch accuracy: 78.9%
Validation accuracy: 87.6%
Minibatch loss at step 3000: 3984.598389
Minibatch accuracy: 83.6%
Validation accuracy: 88.0%
Test accuracy: 93.9%


In [17]:
############################################################
########## Neural Networks with L2 regularization ##########
############################################################
##########           Go Deeper                    ##########   
batch_size = 512
beta = 0.0095
graph = tf.Graph()
keep_prob = 0.8

hidden_layer1 = 512
hidden_layer2 = 256
hidden_layer3 = 64

def forward_prop(dataset, weights1, biases1, weights2, biases2, weights3, biases3, weights4, biases4):
    relus = tf.tanh(tf.matmul(dataset, weights1) + biases1)
    relus = tf.tanh(tf.matmul(relus, weights2) + biases2)
    relus = tf.tanh(tf.matmul(relus, weights3) + biases3)
    result = tf.matmul(relus, weights4) + biases4
    return result

def forward_prop_training(dataset, weights1, biases1, weights2, biases2, weights3, biases3, weights4, biases4):
    
    relus = tf.tanh(tf.matmul(dataset, weights1) + biases1)
    dropped = tf.nn.dropout(relus, keep_prob)
    relus = tf.tanh(tf.matmul(dropped, weights2) + biases2)
    dropped = tf.nn.dropout(relus, keep_prob)
    relus = tf.tanh(tf.matmul(dropped, weights3) + biases3)
    dropped = tf.nn.dropout(relus, keep_prob)    
    result = tf.matmul(dropped, weights4) + biases4
    return result



with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer1]))
  biases1 = tf.Variable(tf.zeros([hidden_layer1]))
    
  weights2 = tf.Variable(tf.truncated_normal([hidden_layer1, hidden_layer2]))
  biases2 = tf.Variable(tf.zeros([hidden_layer2]))


  weights3 = tf.Variable(tf.truncated_normal([hidden_layer2, hidden_layer3]))
  biases3 = tf.Variable(tf.zeros([hidden_layer3]))


  weights4 = tf.Variable(tf.truncated_normal([hidden_layer3, num_labels]))
  biases4 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = forward_prop_training(tf_train_dataset, weights1, biases1, weights2, biases2, weights3, biases3\
                                , weights4, biases4)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) +\
                tf.nn.l2_loss(weights4))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step,decay_steps = 1000, decay_rate = 0.9)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2 \
                                               , weights3, biases3, weights4, biases4))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2 \
                                              , weights3, biases3, weights4, biases4))
    
num_steps = 3001

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions,rate = session.run(
              [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2028.187012
Minibatch accuracy: 10.7%
Validation accuracy: 10.7%
Minibatch loss at step 500: 799.701843
Minibatch accuracy: 61.7%
Validation accuracy: 70.1%
Minibatch loss at step 1000: 332.651550
Minibatch accuracy: 72.1%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 145.127777
Minibatch accuracy: 72.3%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 66.240723
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 31.652435
Minibatch accuracy: 79.5%
Validation accuracy: 82.7%
Minibatch loss at step 3000: 15.825279
Minibatch accuracy: 82.2%
Validation accuracy: 83.4%
Test accuracy: 90.2%


In [24]:
############################################################
########## Neural Networks with L2 regularization ##########
############################################################
##########           Go Deeper                    ##########   
batch_size = 1
beta = 0.0095
graph = tf.Graph()
keep_prob = 0.8

hidden_layer1 = 512
hidden_layer2 = 256
hidden_layer3 = 64

def forward_prop(dataset, weights1, biases1, weights2, biases2, weights3, biases3, weights4, biases4):
    relus = tf.tanh(tf.matmul(dataset, weights1) + biases1)
    relus = tf.tanh(tf.matmul(relus, weights2) + biases2)
    relus = tf.tanh(tf.matmul(relus, weights3) + biases3)
    result = tf.matmul(relus, weights4) + biases4
    return result

def forward_prop_training(dataset, weights1, biases1, weights2, biases2, weights3, biases3, weights4, biases4):
    
    relus = tf.tanh(tf.matmul(dataset, weights1) + biases1)
    dropped = tf.nn.dropout(relus, keep_prob)
    relus = tf.tanh(tf.matmul(dropped, weights2) + biases2)
    dropped = tf.nn.dropout(relus, keep_prob)
    relus = tf.tanh(tf.matmul(dropped, weights3) + biases3)
    dropped = tf.nn.dropout(relus, keep_prob)    
    result = tf.matmul(dropped, weights4) + biases4
    return result



with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer1]))
  biases1 = tf.Variable(tf.zeros([hidden_layer1]))
    
  weights2 = tf.Variable(tf.truncated_normal([hidden_layer1, hidden_layer2]))
  biases2 = tf.Variable(tf.zeros([hidden_layer2]))


  weights3 = tf.Variable(tf.truncated_normal([hidden_layer2, hidden_layer3]))
  biases3 = tf.Variable(tf.zeros([hidden_layer3]))


  weights4 = tf.Variable(tf.truncated_normal([hidden_layer3, num_labels]))
  biases4 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = forward_prop_training(tf_train_dataset, weights1, biases1, weights2, biases2, weights3, biases3\
                                , weights4, biases4)

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + \
         beta * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) +\
                tf.nn.l2_loss(weights4))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step,decay_steps = 1000, decay_rate = 0.9)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset, weights1, biases1, weights2, biases2 \
                                               , weights3, biases3, weights4, biases4))
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset, weights1, biases1, weights2, biases2 \
                                              , weights3, biases3, weights4, biases4))
    
num_steps = 160000

with tf.Session(graph=graph) as session:
    with tf.device("/gpu:0"):
        tf.initialize_all_variables().run()
        print("Initialized")
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions,rate = session.run(
              [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
            if (step % 5000 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2025.014404
Minibatch accuracy: 0.0%
Validation accuracy: 10.3%
Minibatch loss at step 5000: 4.231326
Minibatch accuracy: 100.0%
Validation accuracy: 60.0%
Minibatch loss at step 10000: 7.165417
Minibatch accuracy: 0.0%
Validation accuracy: 69.2%
Minibatch loss at step 15000: 0.591178
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 20000: 2.000950
Minibatch accuracy: 0.0%
Validation accuracy: 78.6%
Minibatch loss at step 25000: 0.430677
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 30000: 0.314437
Minibatch accuracy: 100.0%
Validation accuracy: 81.6%
Minibatch loss at step 35000: 4.510186
Minibatch accuracy: 0.0%
Validation accuracy: 81.3%
Minibatch loss at step 40000: 0.317265
Minibatch accuracy: 100.0%
Validation accuracy: 82.6%
Minibatch loss at step 45000: 0.329022
Minibatch accuracy: 100.0%
Validation accuracy: 82.8%
Minibatch loss at step 50000: 0.297271
Minibatch accuracy: 100.0%
Va